## **Text Classification with Fine Tuning BERT**
### **Apa itu BERT?**
BERT adalah singkatan dari **B**idirectional **E**ncoder **R**epresentations from **T**ransformers. Dari kepanjangannya ada dua hal yang dapat digaris bawahi.
BERT menggunakan salah satu bagian dari transformer, yaitu encoder.
BERT itu bidirectional, artinya dilatih dari dua arah (kiri-kanan dan kanan-kiri).

Ada dua model BERT yang berbeda:

1. BERT Base , yang merupakan model BERT terdiri dari 12 Transformer encoder layers, 12 attention heads, 768 hidden size, dan 110M parameters.
2. BERT Large , yang merupakan model BERT terdiri dari 24 Transformer encoder layers, 16 attention heads, 1024 hidden size, dan 340 parameters.

### **Keuntungan dari Fine Tuning**
Kita akan menggunakan BERT untuk melatih pengklasifikasi teks. Secara khusus, kita akan mengambil model BERT yang telah dilatih sebelumnya, dan melatih model baru untuk tugas klasifikasi kami. Mengapa melakukan ini daripada melatih model pembelajaran mendalam tertentu (CNN, BiLSTM, dll.) yang sangat cocok untuk tugas NLP spesifik yang kita butuhkan?

1. **Pengembangan Lebih Cepat**

    * Pertama, bobot model BERT yang telah dilatih sebelumnya telah mengkodekan banyak informasi tentang bahasa kita. Akibatnya, dibutuhkan lebih sedikit waktu untuk melatih model fine-tuned kami - seolah-olah kami telah melatih lapisan bawah jaringan kami secara ekstensif dan hanya perlu menyetelnya dengan lembut saat menggunakan outputnya sebagai fitur untuk tugas klasifikasi kami. Faktanya, penulis merekomendasikan hanya 2-4 periode pelatihan untuk menyempurnakan BERT pada tugas NLP tertentu (dibandingkan dengan ratusan jam GPU yang diperlukan untuk melatih model BERT asli atau LSTM dari awal!).

2. **Lebih Sedikit Data**

    * Selain itu dan mungkin sama pentingnya, karena bobot yang telah dilatih sebelumnya, metode ini memungkinkan kita untuk menyempurnakan tugas kita pada kumpulan data yang jauh lebih kecil daripada yang diperlukan dalam model yang dibangun dari awal. Kelemahan utama dari model NLP yang dibangun dari awal adalah bahwa kita sering membutuhkan kumpulan data yang sangat besar untuk melatih jaringan kita ke akurasi yang wajar, yang berarti banyak waktu dan energi harus dimasukkan ke dalam pembuatan kumpulan data. Dengan menyempurnakan BERT, kami sekarang dapat melatih model untuk kinerja yang baik pada jumlah data pelatihan yang jauh lebih kecil.

3. **Hasil Lebih Baik**

    * Akhirnya, prosedur fine-tuning sederhana ini (biasanya menambahkan satu lapisan yang terhubung penuh di atas BERT dan pelatihan untuk beberapa zaman) ditunjukkan untuk mencapai hasil seni dengan penyesuaian khusus tugas minimal untuk berbagai tugas: klasifikasi, inferensi bahasa, kesamaan semantik, penjawab pertanyaan, dll. Daripada menerapkan arsitektur khusus dan kadang-kadang-kabur yang terbukti bekerja dengan baik pada tugas tertentu, hanya menyempurnakan BERT terbukti menjadi alternatif yang lebih baik (atau setidaknya sama).



### ***Gunakan GPU untuk melatih model karena model dasar BERT berisi 110 juta parameter.***

`Edit 🡒 Notebook Settings 🡒 Hardware accelerator 🡒 (GPU)`

### **Setup library and utility**

### **Installing Hugging Face Library**

Instal paket [transformers](https://github.com/huggingface/transformers) dari Hugging Face yang akan memberi kita antarmuka pytorch untuk bekerja dengan BERT. (Library ini berisi antarmuka untuk model bahasa pra-latihan lainnya seperti OpenAI's GPT dan GPT-2.) Kami telah memilih antarmuka pytorch karena memberikan keseimbangan yang baik antara API tingkat tinggi (yang mudah digunakan tetapi tidak memberikan wawasan ke dalam cara kerja sesuatu).

Saat ini, library Hugging Face tampaknya menjadi interface untuk pytorch yang paling banyak diterima dan kuat untuk bekerja dengan BERT. Selain mendukung berbagai model transformers pre-trained yang berbeda, perpustakaan juga menyertakan modifikasi pre-built dari model ini yang sesuai dengan tugas spesifik Anda. Misalnya, dalam tugas ini kita akan menggunakan `BertForSequenceClassification`.

Library juga menyertakan task-specific classes untuk klasifikasi token, question answering, next sentence prediciton, dll. Menggunakan kelas yang dibuat sebelumnya ini menyederhanakan proses modifikasi BERT untuk tujuan Anda.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
import random

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, get_linear_schedule_with_warmup

from data_utils import TextClassificationDataset, TextClassificationDataLoader
from metrics import text_classification_metrics_fn

### **Tokenization & Input Formatting**

Kita akan mengubah dataset kami ke dalam format yang dapat dilatih BERT.

### **BERT Tokenizer**

Untuk memasukkan teks kita ke BERT, itu harus dipecah menjadi token, dan kemudian token ini harus dipetakan ke indeksnya dalam kosakata tokenizer.

Tokenisasi harus dilakukan oleh tokenizer yang disertakan dengan BERT--sel di bawah ini akan mengunduhnya. Kita akan menggunakan versi "indobenchmark/indobert-base-p1" di sini.

In [3]:
# Load Tokenizer and Config
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1', do_lower_case=True)
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = TextClassificationDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Loading BERT tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Print the original sentence.
print(' Original: ', "Aku adalah bocah petualang")

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize("Aku adalah bocah petualang"))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Aku adalah bocah petualang")))

 Original:  Aku adalah bocah petualang
Tokenized:  ['aku', 'adalah', 'bocah', 'petualang']
Token IDs:  [304, 154, 10829, 27606]


In [5]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50000, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

**Prepare Dataset & Required Formatting**

*Catatan : Format input ke BERT tampaknya "terlalu ditentukan", kita diminta untuk memberikan sejumlah informasi yang tampaknya berlebihan, atau seperti mereka dapat dengan mudah disimpulkan dari data tanpa kami berikan secara eksplisit dia. Tapi memang begitu, dan kita kira itu akan lebih masuk akal setelah kita memiliki pemahaman yang lebih dalam tentang internal BERT.*

Kita diharuskan untuk:
1. Tambahkan ***Add special tokens*** ke awal dan akhir setiap kalimat.
2. ***Pad & truncate*** semua kalimat menjadi satu panjang konstan.
3. Secara eksplisit membedakan token asli dari token padding dengan "***attention mask***".

### **Special Tokens**

**`[SEP]`**

Di akhir setiap kalimat, kita perlu menambahkan token `[SEP]` khusus.

Token ini adalah artefak two-sentence tasks, di mana BERT diberikan dua kalimat terpisah dan diminta untuk menentukan sesuatu (misalnya, dapatkah jawaban pertanyaan di kalimat A ditemukan di kalimat B?).

**`[CLS]`**

Untuk tugas klasifikasi, kita harus menambahkan token `[CLS]` khusus di awal setiap kalimat.

Token ini memiliki arti khusus. BERT terdiri dari 12 lapisan Transformer. Setiap transformator mengambil daftar penyematan token, dan menghasilkan jumlah penyematan yang sama pada output (tetapi dengan nilai fitur yang berubah).

![Ilustrasi tujuan token CLS](https://drive.google.com/uc?export=view&id=1ck4mvGkznVJfW3hv6GUqcdGepVTOx7HE)

Pada output transformator terakhir (12), *hanya embedding pertama (sesuai dengan token [CLS]) yang digunakan oleh classifier*.

> "Token pertama dari setiap urutan selalu merupakan token klasifikasi khusus (`[CLS]`). Status tersembunyi terakhir
sesuai dengan token ini digunakan sebagai representasi urutan agregat untuk klasifikasi
tugas." (dari [kertas BERT](https://arxiv.org/pdf/1810.04805.pdf))

### **Sentence Length & Attention Mask**

Kalimat dalam dataset kita jelas memiliki panjang yang bervariasi, jadi bagaimana BERT menangani ini?

BERT memiliki dua kendala:
1. Semua kalimat harus diisi atau dipotong menjadi satu panjang yang tetap.
2. Panjang kalimat maksimum adalah 512 token.

Padding dilakukan dengan token `[PAD]` khusus, yang berada pada indeks 0 dalam kosakata BERT. Ilustrasi di bawah ini menunjukkan padding ke "MAX_LEN" dari 8 token.

<img src="https://drive.google.com/uc?export=view&id=1cb5xeqLu_5vPOgs3eRnail2Y00Fl2pCo" width="600">

"Attention Mask" merupakan sebuah array dari 1 dan 0 yang menunjukkan token mana yang diisi dan mana yang tidak. Mask ini memberi tahu mekanisme "Self-Attention" di BERT untuk tidak memasukkan token PAD ini ke dalam interpretasi kalimatnya.

In [7]:
train_dataset_path = "dataset/data_worthcheck/train.csv"
dev_dataset_path = "dataset/data_worthcheck/dev.csv"
test_dataset_path = "dataset/data_worthcheck/test.csv"

### **Tokenize Dataset**

Library transformers menyediakan fungsi `encode` yang membantu yang akan menangani sebagian besar langkah-langkah penguraian dan persiapan data untuk kita.

Pada fungsi `TextClassificationDataset` dan `TextClassificationDataLoader` berisikan fungsi-fungsi dan metode untuk melakukan encoding, menambahkan special tokens, memberikan padding, dan menentukan mana yang termasuk attention mask.

In [8]:
train_dataset = TextClassificationDataset(train_dataset_path, tokenizer)
dev_dataset = TextClassificationDataset(dev_dataset_path, tokenizer)
test_dataset = TextClassificationDataset(test_dataset_path, tokenizer)

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, recommended a batch are size of 16 or 32.

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_loader = TextClassificationDataLoader(train_dataset, max_len=512, batch_size=16, shuffle=True)
dev_loader = TextClassificationDataLoader(dev_dataset, max_len=512, batch_size=16, shuffle=False)
test_loader = TextClassificationDataLoader(test_dataset, max_len=512, batch_size=16, shuffle=False)

In [9]:
w2i, i2w = TextClassificationDataset.LABEL2INDEX, TextClassificationDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'no': 0, 'yes': 1}
{0: 'no', 1: 'yes'}


### **TESTING MODEL ON SENTENCE IN DATASET**

In [10]:
text = train_dataset.__getitem__(3)[2] 
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label: {i2w[label]} ({F.softmax(logits, dim=1).squeeze()[label]*100:.2f}%)')

Text: neng solo wes ono terduga corona cobo neng ati mu neng conora | Label: no (51.66%)


### **Optimizer & Learning Rate Scheduler**

Untuk tujuan penyempurnaan, penulis merekomendasikan untuk memilih dari nilai-nilai berikut (dari Lampiran A.3 dari [kertas BERT](https://arxiv.org/pdf/1810.04805.pdf)):

>- **Ukuran batch:** 16, 32
- **Kecepatan pembelajaran (Adam):** 5e-5, 3e-5, 2e-5
- **Jumlah epoch:** 2, 3, 4

Kami memilih:
* Ukuran batch: 16 (diatur saat membuat DataLoaders)
* Tingkat pembelajaran: 2e-5
* Epochs: 2

In [11]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)
model = model.cuda()

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Set the seed value all over the place to make this reproducible.
seed_val = 24092022

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

### **Training Loop**

**Pelatihan:**
- Unpack input dan label data
- Load data ke GPU untuk akselerasi
- Hapus gradien yang dihitung di lintasan sebelumnya.
     - Di pytorch, gradien terakumulasi secara default (berguna untuk hal-hal seperti RNN) kecuali Anda menghapusnya secara eksplisit.
- Forward pass
- Backward pass
- Beri tahu jaringan untuk memperbarui parameter dengan optimizer.step()
- Lacak variabel untuk memantau kemajuan progress

**Evaluasi:**
- Unpack input dan label data
- Load data ke GPU untuk akselerasi
- Forward pass
- Hitung kerugian pada data validasi kami dan lacak variabel untuk memantau kemajuan progress

In [12]:
#TRAINING MODEL

# ========================================
#               Training
# ========================================
def train(model, train_loader, dev_loader, optimizer, device, epochs=2):
    model.to(device)
    model.train()
    best_acc = 0
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)

        # Reset the total loss for this epoch.
        train_loss = 0
        train_acc = 0
        train_steps = 0

        # For each batch of training data...
        for batch in tqdm(train_loader):
            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            optimizer.zero_grad()
            loss, _, _ = forward_sequence_classification(model, batch[:-1], i2w=i2w, device="cuda")

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            train_loss += loss.item()
            train_steps += 1

        train_loss /= train_steps
        train_metrics = evaluate(model, train_loader, device)
        dev_metrics = evaluate(model, dev_loader, device)
        train_acc = train_metrics['accuracy']
        train_f1 = train_metrics['f1']
        train_precision = train_metrics['precision']
        train_recall = train_metrics['recall']
        print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1: {train_f1*100:.2f}% | Train Precision: {train_precision*100:.2f}% | Train Recall: {train_recall*100:.2f}%')
      
def evaluate(model, data_loader, device):
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    y_true = []
    y_pred = []

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():
        for batch in data_loader:
            _, y_true_batch, y_pred_batch = forward_sequence_classification(model, batch[:-1], i2w=i2w, device="cuda")
            y_true.extend(y_true_batch)
            y_pred.extend(y_pred_batch)
    # model.train()
    return text_classification_metrics_fn(y_pred, y_true)

def forward_sequence_classification(model, batch, i2w, device):
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using the 
    # `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    input_ids, attention_mask, labels = batch
    input_ids = torch.IntTensor(input_ids).to(device)
    attention_mask = torch.IntTensor(attention_mask).to(device)
    labels = torch.LongTensor(labels).to(device)

    output = model(input_ids, attention_mask=attention_mask, labels=labels)
    # Get the loss and "logits" output by the model. The "logits" are the 
    # output values prior to applying an activation function like the 
    # softmax.
    loss, logits = output[:2]
    
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, k=1, dim=-1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[labels[j][0].item()])
    
    return loss, list_label, list_hyp

model_result = train(model, train_loader, dev_loader, optimizer, device="cuda", epochs=2)
model_result

Epoch 1/2
----------


100%|██████████| 1351/1351 [05:00<00:00,  4.50it/s]


Train Loss: 0.278 | Train Acc: 93.80% | Train F1: 92.61% | Train Precision: 91.36% | Train Recall: 94.22%
Epoch 2/2
----------


100%|██████████| 1351/1351 [04:57<00:00,  4.54it/s]


Train Loss: 0.132 | Train Acc: 97.79% | Train F1: 97.28% | Train Precision: 97.12% | Train Recall: 97.44%


In [13]:
#Evaluate on test
model.eval()
torch.set_grad_enabled(False)

# Tracking variables 
total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader)
for batch in pbar:
    loss, y_true, y_pred = forward_sequence_classification(model, batch[:-1], i2w=i2w, device="cuda")
    total_loss += loss.item()
    list_hyp.extend(y_pred)
    list_label.extend(y_true)

#Save Prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('result.txt', index=False)
print(df)

100%|██████████| 175/175 [00:11<00:00, 15.78it/s]

      index label
0         0    no
1         1    no
2         2    no
3         3    no
4         4    no
...     ...   ...
2795   2795    no
2796   2796    no
2797   2797   yes
2798   2798    no
2799   2799   yes

[2800 rows x 2 columns]


Test fine-tuned model on sample sentences using Test Sample

In [14]:
#Get metrics for test data
test_metrics = text_classification_metrics_fn(list_hyp, list_label)

print(f'Test Loss: {total_loss/len(test_loader):.3f} | Test Acc: {test_metrics["accuracy"]*100:.2f}% | Test F1: {test_metrics["f1"]*100:.2f}% | Test Precision: {test_metrics["precision"]*100:.2f}% | Test Recall: {test_metrics["recall"]*100:.2f}%')

Test Loss: 0.295 | Test Acc: 89.04% | Test F1: 85.58% | Test Precision: 85.34% | Test Recall: 85.83%


In [15]:
text = test_dataset.__getitem__(0)[2]
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: jek dajal ga depok bang | Label : no (99.918%)
